In [ ]:
import json, time, re
from enum import Enum

In [ ]:

class LogLevel(Enum):
     Info = 1
     Warn = 2
     Exception = 3

class Logger:
    def __init__(self, Level):
        self.__logs = []
        self.__LogLevel = Level
    def LogInfo(self, message):
        self.__logs.append ({'LogLevel': LogLevel.Info.value, 'Message':message})  
    def LogWarn(self, message):
        self.__logs.append ({'LogLevel': LogLevel.Warn.value, 'Message':message}) 
    def LogException(self, message):
        self.__logs.append ({'LogLevel': LogLevel.Exception.value, 'Message':message})  
    def get_log(self):
        logs = [l for l in self.__logs if l['LogLevel'] >= self.__LogLevel.value ]
        return logs  
    def PrettyPrint(self): 
        for d in self.get_log():
            print(f" {LogLevel(d['LogLevel'])}: {d['Message']} " )
    def to_json(self, path):  
         with open(f'{path}', 'w') as f:
            json.dump(self.get_log(), f)

In [25]:

s = time.time()
time.sleep(.25)

logger = Logger(LogLevel.Info) 
logger.LogInfo('trying')
try: 
    logger.LogInfo('open connection')
    logger.LogWarn('warning')
    err    
except Exception as e:
    logger.LogException(f'Exception {e}')          
finally: 
    logger.LogInfo('connection closed')     
        
logger.PrettyPrint()     
logger.to_json('C:\\temp\\log.json')

e = time.time()
print(e-s)


 LogLevel.Info: trying 
 LogLevel.Info: open connection 
 LogLevel.Warn: warning 
 LogLevel.Exception: Exception name 'err' is not defined 
 LogLevel.Info: connection closed 
0.2571704387664795
